In [1]:
path = 'https://www.gstatic.com/covid19/mobility/Global_Mobility_Report.csv'

In [2]:
!pip install modin[dask]
!pip install polars
!pip install pyarrow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 868.9/868.9 kB 17.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 20.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 830.7/830.7 kB 15.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.4/322.4 kB 20.7 MB/s eta 0:00:00
--- Logging error ---
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/pip/_internal/utils/logging.py", line 177, in emit
    self.console.print(renderable, overflow="ignore", crop=False, style=style)
  File "/usr/local/lib/python3.9/dist-packages/pip/_vendor/rich/console.py", line 1673, in print
    extend(render(renderable, render_options))
  File "/usr/local/lib/python3.9/dist-packages/pip/_vendor/rich/console.py", line 1305, in render
    for render_output in iter_render:
  File "/usr/local/lib/python3.9/dist-packages/pip/_inte

In [3]:
!wget -r -nc https://www.gstatic.com/covid19/mobility/Global_Mobility_Report.csv

--2022-08-04 16:29:24--  https://www.gstatic.com/covid19/mobility/Global_Mobility_Report.csv
Resolving www.gstatic.com (www.gstatic.com)... 142.251.40.227, 2607:f8b0:4006:823::2003
Connecting to www.gstatic.com (www.gstatic.com)|142.251.40.227|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1056783891 (1008M) [text/csv]
Saving to: ‘www.gstatic.com/covid19/mobility/Global_Mobility_Report.csv’

www.gstatic.com/cov 100%[===================>]   1008M  19.8MB/s    in 53s     

2022-08-04 16:30:17 (18.9 MB/s) - ‘www.gstatic.com/covid19/mobility/Global_Mobility_Report.csv’ saved [1056783891/1056783891]

FINISHED --2022-08-04 16:30:17--
Total wall clock time: 54s
Downloaded: 1 files, 1008M in 53s (18.9 MB/s)


In [4]:
import polars as pl
import pyarrow as pa
import pandas as pd
import dask
import dask.dataframe as dd
import modin.pandas as md

In [5]:
pd.read_csv('Global_Mobility_Report.csv', nrows=3)

,country_region_code,country_region,sub_region_1,sub_region_2,metro_area,iso_3166_2_code,census_fips_code,place_id,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
0,AE,United Arab Emirates,NaN,NaN,NaN,NaN,NaN,ChIJvRKrsd9IXj4RpwoIwFYv0zM,2020-02-15,0,4,5,0,2,1
1,AE,United Arab Emirates,NaN,NaN,NaN,NaN,NaN,ChIJvRKrsd9IXj4RpwoIwFYv0zM,2020-02-16,1,4,4,1,2,1
2,AE,United Arab Emirates,NaN,NaN,NaN,NaN,NaN,ChIJvRKrsd9IXj4RpwoIwFYv0zM,2020-02-17,-1,1,5,1,2,1


In [6]:
pd.read_csv('Global_Mobility_Report.csv', nrows=3).columns

Index(['country_region_code', 'country_region', 'sub_region_1', 'sub_region_2',
       'metro_area', 'iso_3166_2_code', 'census_fips_code', 'place_id', 'date',
       'retail_and_recreation_percent_change_from_baseline',
       'grocery_and_pharmacy_percent_change_from_baseline',
       'parks_percent_change_from_baseline',
       'transit_stations_percent_change_from_baseline',
       'workplaces_percent_change_from_baseline',
       'residential_percent_change_from_baseline'],
      dtype='object')

In [7]:
columns = ['country_region_code', 'country_region', 'date',
       'retail_and_recreation_percent_change_from_baseline',
       'grocery_and_pharmacy_percent_change_from_baseline',
       'parks_percent_change_from_baseline',
       'transit_stations_percent_change_from_baseline',
       'workplaces_percent_change_from_baseline',
       'residential_percent_change_from_baseline']

In [8]:
# dd.read_csv('Global_Mobility_Report.csv').to_parquet('mobility.parquet',compression='snappy',engine='pyarrow', low_memory=False)
(dd.read_csv('Global_Mobility_Report.csv',
            assume_missing=True, 
            usecols= columns,
            parse_dates=['date'])
   .head())

,country_region_code,country_region,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
0,AE,United Arab Emirates,2020-02-15,0.0,4.0,5.0,0.0,2.0,1.0
1,AE,United Arab Emirates,2020-02-16,1.0,4.0,4.0,1.0,2.0,1.0
2,AE,United Arab Emirates,2020-02-17,-1.0,1.0,5.0,1.0,2.0,1.0
3,AE,United Arab Emirates,2020-02-18,-2.0,1.0,5.0,0.0,2.0,1.0
4,AE,United Arab Emirates,2020-02-19,-2.0,0.0,4.0,-1.0,2.0,1.0


In [9]:
(dd.read_csv('Global_Mobility_Report.csv',
            assume_missing=True, 
            usecols= columns,
            parse_dates=['date'])
).to_parquet('mobility.parquet',compression='snappy',engine='pyarrow')

(None,)

In [10]:
!ls

Global_Mobility_Report.csv  dask-worker-space  www.gstatic.com
Test.ipynb		    mobility.parquet


In [11]:
dddf = dd.read_parquet('mobility.parquet', engine='pyarrow')

In [12]:
from distributed import Client
client = Client()

distributed.diskutils - INFO - Found stale lock file and directory '/notebooks/dask-worker-space/worker-nv2ks4cc', purging
distributed.diskutils - INFO - Found stale lock file and directory '/notebooks/dask-worker-space/worker-4qq98ufy', purging
distributed.diskutils - INFO - Found stale lock file and directory '/notebooks/dask-worker-space/worker-i57w5rzb', purging
/usr/lib/python3.9/contextlib.py:126: UserWarning: Creating scratch directories is taking a surprisingly long time. This is often due to running workers on a network file system. Consider specifying a local-directory to point workers to write scratch data to a local disk.
  next(self.gen)


In [13]:
mddf = md.read_parquet('mobility.parquet', engine='pyarrow')

In [14]:
pdf = pd.read_parquet('mobility.parquet', engine='pyarrow')

In [15]:
pldf = pl.read_parquet('mobility.parquet', use_pyarrow=True)

## Read Shape

In [16]:
%time dddf.shape[0].compute()

CPU times: user 251 ms, sys: 51.8 ms, total: 303 ms
Wall time: 1.67 s


10806198

In [17]:
%time pdf.shape

CPU times: user 32 µs, sys: 0 ns, total: 32 µs
Wall time: 39.3 µs


(10806198, 9)

In [18]:
%time mddf.shape

CPU times: user 0 ns, sys: 55 µs, total: 55 µs
Wall time: 54.8 µs


(10806198, 9)

In [19]:
%time pldf.shape

CPU times: user 42 µs, sys: 0 ns, total: 42 µs
Wall time: 48.4 µs


(10806198, 10)

Polars is fastest, then pandas < modin < dask

## Country Region Count

In [38]:
%time dddf['country_region'].value_counts().nlargest(10).compute()

CPU times: user 99.6 ms, sys: 14.1 ms, total: 114 ms
Wall time: 720 ms


United States     2312412
Brazil            1808620
India              598225
Turkey             493512
Argentina          378917
United Kingdom     374096
Poland             346884
Netherlands        319961
Colombia           286514
Australia          250888
Name: country_region, dtype: int64

In [39]:
%time pdf['country_region'].value_counts().nlargest(10)

CPU times: user 434 ms, sys: 0 ns, total: 434 ms
Wall time: 424 ms


United States     2312412
Brazil            1808620
India              598225
Turkey             493512
Argentina          378917
United Kingdom     374096
Poland             346884
Netherlands        319961
Colombia           286514
Australia          250888
Name: country_region, dtype: int64

In [40]:
%time mddf['country_region'].value_counts().nlargest(10)

CPU times: user 424 ms, sys: 49.4 ms, total: 474 ms
Wall time: 932 ms


United States     2312412
Brazil            1808620
India              598225
Turkey             493512
Argentina          378917
United Kingdom     374096
Poland             346884
Netherlands        319961
Colombia           286514
Australia          250888
Name: country_region, dtype: int64

In [26]:
%time pldf.groupby('country_region').count().sort(by = 'count', reverse=True).head(10)

CPU times: user 634 ms, sys: 198 ms, total: 833 ms
Wall time: 227 ms


country_region,count
str,u32
"""United States""",2312412
"""Brazil""",1808620
"""India""",598225
"""Turkey""",493512
"""Argentina""",378917
"""United Kingdom...",374096
"""Poland""",346884
"""Netherlands""",319961
"""Colombia""",286514


In [20]:
%time mddf.groupby('country_region')['country_region'].count().sort_values(ascending=False).head(10)

Please refer to https://modin.readthedocs.io/en/stable/supported_apis/defaulting_to_pandas.html for explanation.


CPU times: user 712 ms, sys: 116 ms, total: 828 ms
Wall time: 3.54 s


country_region
United States     2312412
Brazil            1808620
India              598225
Turkey             493512
Argentina          378917
United Kingdom     374096
Poland             346884
Netherlands        319961
Colombia           286514
Australia          250888
Name: country_region, dtype: int64

In [21]:
%time dddf.groupby(['country_region',dddf.date.dt.month_name() ])['grocery_and_pharmacy_percent_change_from_baseline'].mean().compute()

CPU times: user 568 ms, sys: 63.2 ms, total: 631 ms
Wall time: 3.91 s


country_region  date     
Afghanistan     April        59.950000
                August       56.489362
                December     64.064516
                February     54.802817
                January      60.564516
                               ...    
Zimbabwe        March        43.189616
                May          47.511468
                November     47.378092
                October      46.393220
                September    40.555133
Name: grocery_and_pharmacy_percent_change_from_baseline, Length: 1620, dtype: float64

In [22]:
%time pdf.groupby(['country_region',pdf.date.dt.month_name() ])['grocery_and_pharmacy_percent_change_from_baseline'].mean()

CPU times: user 5.43 s, sys: 1.29 s, total: 6.72 s
Wall time: 6.4 s


country_region  date     
Afghanistan     April        59.950000
                August       56.489362
                December     64.064516
                February     54.802817
                January      60.564516
                               ...    
Zimbabwe        March        43.189616
                May          47.511468
                November     47.378092
                October      46.393220
                September    40.555133
Name: grocery_and_pharmacy_percent_change_from_baseline, Length: 1620, dtype: float64

In [23]:
mddf.groupby(['country_region',mddf.date.dt.month_name() ])['grocery_and_pharmacy_percent_change_from_baseline'].mean()

NotImplementedError: Column lookups on GroupBy with arbitrary Series in by is not yet supported.

In [24]:
%%time
(mddf[['country_region','date', 'grocery_and_pharmacy_percent_change_from_baseline']]
 .assign(month = mddf.date.dt.month_name())
 .drop('date',axis=1)
).groupby(['country_region','month'])['grocery_and_pharmacy_percent_change_from_baseline'].mean()

CPU times: user 1.58 s, sys: 300 ms, total: 1.88 s
Wall time: 13 s


country_region  month    
Afghanistan     April        59.950000
                August       56.489362
                December     64.064516
                February     54.802817
                January      60.564516
                               ...    
Zimbabwe        March        43.189616
                May          47.511468
                November     47.378092
                October      46.393220
                September    40.555133
Name: grocery_and_pharmacy_percent_change_from_baseline, Length: 1620, dtype: float64

In [27]:
month_names = {"1":"January", "2":"February", "3":"March","4":"April","5":"May","6":"June","7":"July","8":"August","9":"September","10":"October","11":"November","12":"December"}
month_names

{'1': 'January',
 '2': 'February',
 '3': 'March',
 '4': 'April',
 '5': 'May',
 '6': 'June',
 '7': 'July',
 '8': 'August',
 '9': 'September',
 '10': 'October',
 '11': 'November',
 '12': 'December'}

In [44]:
%%time
(
(pldf[['country_region','date', 'grocery_and_pharmacy_percent_change_from_baseline']]
 .with_column(pldf['date'].dt.month()
              .cast(str)
              .apply(lambda month:month_names.get(month)))
 
)
    .groupby(['country_region','date']).agg([pl.col('grocery_and_pharmacy_percent_change_from_baseline').mean().alias("mean")])
    .sort(by=["country_region","date"])
)

CPU times: user 7.22 s, sys: 1.59 s, total: 8.81 s
Wall time: 7.1 s


country_region,date,mean
str,str,f64
"""Afghanistan""","""April""",59.95
"""Afghanistan""","""August""",56.489362
"""Afghanistan""","""December""",64.064516
"""Afghanistan""","""February""",54.802817
"""Afghanistan""","""January""",60.564516
"""Afghanistan""","""July""",85.522222
"""Afghanistan""","""June""",118.475
"""Afghanistan""","""March""",70.295699
"""Afghanistan""","""May""",87.64375
